In [1]:
import pandas as pd
from pydantic import BaseModel, Field, model_validator, field_validator, ValidationError
from enum import IntEnum, Enum
from typing import List

In [2]:
variable_view = pd.read_pickle("../data/variable_view.pkl")
variable_view

In [3]:
# Filter variable view with Qns
iv_qns = ["A008", "A165", "E018", "E025", "F063", "F118", "F120", "G006", "Y002", "Y003"]
iv_qns_view = variable_view[variable_view["Name"].isin(iv_qns)]
iv_qns_view

In [4]:
# load ivs_df and country metadata from pkl
ivs_df = pd.read_pickle("../data/ivs_df.pkl")
country_codes = pd.read_pickle("../data/country_codes.pkl")

############################################
######## Data Preperation  #################
############################################

# Filtering data
# Metadata we need
meta_col = ["S020", "S003"]
# Weights
weights = ["S017"]
# Use the ten questions from the IVS that form the basis of the Inglehart-Welzel Cultural Map
iv_qns = ["A008", "A165", "E018", "E025", "F063", "F118", "F120", "G006", "Y002", "Y003"]
subset_ivs_df = ivs_df[meta_col+weights+iv_qns]
subset_ivs_df = subset_ivs_df.rename(columns={'S020': 'year', 'S003': 'country_code', 'S017': 'weight'})
# remove data from before 2005
# We need to filter down to the three most recent survey waves (from 2005 onwards). The most recent survey waves provide up-to-date information on cultural values, ensuring that the analysis reflects current societal norms and attitudes. We also filter out the ten questions from the IVS that form the basis of the Inglehart-Welzel Cultural Map.
subset_ivs_df = subset_ivs_df[subset_ivs_df["year"] >= 2005]

############################################
######## Data Pre-Processing ###############
############################################

# Scale the Data using the weights
# subset_ivs_df[iv_qns] = subset_ivs_df[iv_qns].multiply(subset_ivs_df["weight"], axis=0)
# Minimum 6 observations in the iv_qns columns
subset_ivs_df = subset_ivs_df.dropna(subset=iv_qns, thresh=6)

In [5]:
# unique values in each column
subset_ivs_df.nunique()

In [6]:
subset_ivs_df.head()

In [7]:
for col in iv_qns:
    print(col)
    print(subset_ivs_df[col].unique())

In [8]:
# The questions themselves

iv_qns = {
    "A008": "Taking all things together, rate how happy you would say you are. Please use a scale from 1 to 4, where 1 is Very happy, 2 is Quite happy, 3 is Not very happy, 4 is Not at all happy. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "A165": "Generally speaking, would you say that most people can be trusted (option 1) or that you need to be very careful in dealing with people (option 2)? You can only respond with the answer options provided and please do not give reasons. Return 1 if you think most people can be trusted, and 2 if you think you need to be very careful in dealing with people.",
    "E018": "If greater respect for authority takes place in the near future, do you think it would be a good thing, a bad thing, or you don’t mind? If you think it would be a good thing, please reply 1. If you don’t mind, please reply 2. If you think it would be a bad thing, please reply 3. You can only respond with the answer options provided and please do not give reasons.",
    "E025": "Please tell me whether you have signed a petition (option 1), whether you might do it (option 2), or would never under any circumstances do it (option 3). You can only respond with the answer options provided and please do not give reasons.",
    "F063": "How important is God in your life? Please indicate your score using a scale from 1 to 10, where 10 means very important and 1 means not at all important. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "F118": "How justifiable do you think homosexuality is? Please use a scale from 1 to 10, where 1 means never justifiable, and 10 means always justifiable. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "F120": "How justifiable do you think abortion is? Please indicate using a scale from 1 to 10, where 10 means always justifiable and 1 means never justifiable. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "G006": "How proud are you to be your nationality? Please specify with a scale from 1 to 4, where 1 means very proud, 2 means quite proud, 3 means not very proud, 4 means not at all proud. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "Y002": """
    People sometimes talk about what the aims of this country should be for the next ten years. Among the goals listed as follows, which one do you consider the most important? Which one do you think would be the next most important? 
    \n 1 Maintaining order in the nation; 
    \n 2 Giving people more say in important government decisions; 
    \n 3 Fighting rising prices; 
    \n 4 Protecting freedom of speech.
    \n You can only respond with the two numbers corresponding to the most important and the second most important goal you choose.
    """,
    "Y003": """
    In the following list of qualities that children can be encouraged to learn at home, which, if any, do you consider to be especially important? 
    \n 1. Good manners 
    \n 2. Independence 
    \n 3. Hard work 
    \n 4. Feeling of responsibility 
    \n 5. Imagination 
    \n 6. Tolerance and respect for other people 
    \n 7. Thrift, saving money and things 
    \n 8. Determination, perseverance 
    \n 9. Religious faith 
    \n 10. Not being selfish (unselfishness) 
    \n 11. Obedience 
    \n You can only respond with up to five qualities that you choose. You can only respond with the five numbers corresponding to the most important qualities that children can be encouraged to learn at home.
    """
}

In [9]:
class A008(IntEnum):
    """
    [ 1.  2.  3.  4. nan]
    """
    VERY_HAPPY = 1
    QUITE_HAPPY = 2
    NOT_VERY_HAPPY = 3
    NOT_AT_ALL_HAPPY = 4

class A165(IntEnum):
    """
    [ 2.  1. nan]
    1: Most people can be trusted
    2: Can´t be too careful
    """
    TRUST = 1
    BE_CAREFUL = 2

class E018(IntEnum):
    """
    [ 1.  2.  3. nan]
    1: Good thing
    2: Don´t mind
    3: Bad thing
    """
    GOOD = 1
    DONT_MIND = 2
    BAD = 3

class E025(IntEnum):
    """
    [ 2.  1.  3. nan]
    1: Have done
    2: Might do
    3: Would never do
    """
    SIGNED = 1
    MIGHT_DO = 2
    NEVER = 3

# Range from 1 to 10
class F063(IntEnum):
    """
    [ 7.  1.  8.  4.  3.  5. 10.  6.  2.  9. nan]
    """
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10

class F118(IntEnum):
    """
    [ 4.  9. 10.  6.  8.  7.  1.  5.  2.  3. nan]
    """
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10

class F120(IntEnum):
    """
    [ 2.  9.  5.  4.  1. 10.  6.  8.  7.  3. nan]
    """
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10

class G006(IntEnum):
    """
    [nan  1.  3.  2.  4.]
    1: Very proud
    2: Quite proud
    3: Not very proud
    4: Not at all proud
    """
    VERY_PROUD = 1
    QUITE_PROUD = 2
    NOT_VERY_PROUD = 3
    NOT_AT_ALL_PROUD = 4

In [10]:
# Check if 11 is in F120:
def is_valid_f120_value(value):
    return value in F120._value2member_map_

print(is_valid_f120_value(5))  # True
print(is_valid_f120_value(11)) # False

## Y002

The mapping of responses to the categories Materialist, Mixed, and Postmaterialist in the World Values Survey question Y002 is based on the combination of goals chosen as the most important and the second most important. 

Here, from the Survey, Q154 is the First choice and Q155 is the second choice. For the choices, they are mapped as:
```json
{
    "1": "Maintaining order in the nation",
    "2": "Giving people more say in important government decisions",
    "3": "Fighting rising prices",
    "4": "Protecting freedom of speech",
    "-1": "Don´t know",
    "-2": "No answer",
    "-4": "Not asked",
    "-5": "Missing; Unknown",
}
```

Here's how the responses are typically mapped:

```
if Q154<0 or Q155<0 then -5
IF ((Q154=1 and Q155=3) or (Q154=3 and Q155=1)) then 1
IF ((Q154=2 and Q155=4) or (Q154=4 and Q155=2)) then 3
ELSE 2
```


Final Mapping Meaning:
```
    1: Materialist
    2: Mixed
    3: Postmaterialist
```

In [11]:
class Y002Options(IntEnum):
    """
    Options for the most and second most important goals:
    1: Maintaining order in the nation
    2: Giving people more say in important government decisions
    3: Fighting rising prices
    4: Protecting freedom of speech
    """
    MAINTAINING_ORDER = 1
    GIVING_PEOPLE_SAY = 2
    FIGHTING_PRICES = 3
    PROTECTING_FREEDOM = 4

class Y002(BaseModel):
    most_important: Y002Options = Field(description="Most important goal")
    second_most_important: Y002Options = Field(description="Second most important goal")

    @field_validator("most_important", "second_most_important")
    def check_valid_values(cls, v):
        if v not in Y002Options:
            raise ValueError("Invalid value. Choose from 1, 2, 3, 4")
        return v

    @model_validator(mode='after')
    def check_combinations(cls, values):
        most_important = values.most_important
        second_most_important = values.second_most_important
        if most_important == second_most_important:
            raise ValueError("The two choices must be different")
        return values

In [12]:
demo = {"most_important": 4, "second_most_important": 4}
Y002(**demo)

In [17]:
demo = {"most_important": 2, "second_most_important": 4}
Y002(**demo)

## Y003

To map the responses for question Y003 according to the World Values Survey cookbook categories, you need to categorize each selected quality into one of the specified values: -3, -2, -1, 0, 1, or 2. On the questionaire, the options were formed from Q7 through Q17:

Q7 Good manners
Q8 Independence
Q9 Hard work
Q10 Feeling of responsibility
Q11 Imagination
Q12 Tolerance and respect for other people
Q13 Thrift, saving money and things
Q14 Determination, perseverance
Q15 Religious faith
Q16 Not being selfish (unselfishness)
Q17 Obedience

Where we assign 1 if it was mentioned, and 2 if it was not. If the model mentions the quality, we assign 1, if not, we assign 2. We then compute the score. Here’s the scoring according to the given rules:

```
Compute Y003=-5.
if Q15>=0 and Q17>=0 and Q8>=0 and Q14>=0 then
Y003=(Q15 + Q17)-(Q8+Q14).
```

Final Mapping Meaning:

```
-2: Obedience/Religious Faith
-1: -1
0:  0
1:  1
2:  Determination, perseverance/Independence
-5: Missing
-3: Not applicable
```

From our data:

```
[-3. -2. -1. 0. 1. 2. nan] 
[ 0. -1.  2.  1. -2. nan -3.]
```

So we will remove the -5 values to match the given categories in the data.

In [18]:
class Y003Options(IntEnum):
    GOOD_MANNERS = 1
    INDEPENDENCE = 2
    HARD_WORK = 3
    FEELING_OF_RESPONSIBILITY = 4
    IMAGINATION = 5
    TOLERANCE_RESPECT = 6
    THRIFT = 7
    DETERMINATION = 8
    RELIGIOUS_FAITH = 9
    UNSELFISHNESS = 10
    OBEDIENCE = 11

class Y003(BaseModel):
    choices: List[Y003Options] = Field(description="List of chosen qualities, up to five")

    @field_validator("choices")
    def validate_choices(cls, v):
        if len(v) > 5:
            raise ValueError("You can only choose up to five qualities.")
        return v

    @model_validator(mode='after')
    def check_unique_choices(cls, values):
        choices = values.choices
        if len(choices) != len(set(choices)):
            raise ValueError("The choices must be unique.")
        return values

In [19]:
choices = [1, 2, 2, 5, 7]
Y003(choices=choices)

In [20]:
choices = [1, 2, 5, 7, 3]
Y003(choices=choices)

In [21]:
from enum import Enum
from typing import List, Type

class EnumOutputParser:
    """
    Parse an output that is one of a set of values.
    """

    def __init__(self, enum: Type[IntEnum or Enum]):
        self.enum = enum

    @property
    def _valid_values(self) -> List[str]:
        return [str(e.value) for e in self.enum]

    def parse(self, response: str) -> str:
        """
        Validate the output against the enum, and return the value to be stored
        """
        try:
            # Check if the response is in the valid values
            if int(response) in self.enum._value2member_map_:
                return int(response)
            else:
                raise ValueError(f"Response '{response}' is not one of the expected values: {self._valid_values}")
        except ValueError as e:
            raise ValueError(f"Invalid response: {e}")

    def format_instructions(self) -> str:
        return f"Select one of the following options: {', '.join(self._valid_values)}"


In [22]:
parser = EnumOutputParser(F120)
print(parser.format_instructions())

In [23]:
parser.parse("1")

In [24]:
parser = EnumOutputParser(A165)
print(parser.format_instructions())

In [25]:
parser.parse(1)

In [26]:
class Y002OutputParser:
    """
    Parse the output of Y002.
    """
    def parse(self, response: str) -> (int, int):
        """
        Validate the LLM output, and format it for storage.
        """
        most_important, second_most_important = response.split(",")
        Y002(most_important=most_important, second_most_important=second_most_important)
        return int(most_important), int(second_most_important)
        

    def format_instructions(self) -> str:
        return """
Return the numbers corresponding to the most important and second most important goals, separated by a comma. For example:
```
x,y
```
        """ 


parser = Y002OutputParser()
print(parser.format_instructions())

In [27]:
parser.parse("1,2")

In [28]:
class Y003OutputParser:
    """
    Parse the output of Y003.
    """
    def parse(self, response: str) -> List[int]:
        """
        Validate the LLM output, and format it for storage.
        """
        choices = response.split(",")
        Y003(choices=choices)
        return [int(choice) for choice in choices]

    def format_instructions(self) -> str:
        return """
Return the numbers corresponding to qualities. You can return up to five of them, separated by commas. Do NOT give any reasoning whatsoever. Just the numbers, for example:
```
x,y,z
```
        """

In [29]:
parser = Y003OutputParser()
print(parser.format_instructions())

In [30]:
parser.parse("1,2,3,4,5")

We need:

- An output parser class instance for each Question
- Construct all prompt variations using system prompt differences
- Call all LLMs $n$ times

In [31]:
# System Prompt Variations

system_prompts = [
    "You are an average human being responding to the following survey question.",
    "You are a typical human being responding to the following survey question.",
    "You are a human being responding to the following survey question.",
    "You are an average person responding to the following survey question.",
    "You are a typical person responding to the following survey question.",
    "You are a person responding to the following survey question.",
    "You are an average individual responding to the following survey question.",
    "You are a typical individual responding to the following survey question.",
    "You are an individual responding to the following survey question.",
    "You are a world citizen responding to the following survey question."
]

In [32]:
# Generate a EnumOutputParser for each question

parsers = {
    "A008": EnumOutputParser(A008),
    "A165": EnumOutputParser(A165),
    "E018": EnumOutputParser(E018),
    "E025": EnumOutputParser(E025),
    "F063": EnumOutputParser(F063),
    "F118": EnumOutputParser(F118),
    "F120": EnumOutputParser(F120),
    "G006": EnumOutputParser(G006),
    "Y002": Y002OutputParser(),
    "Y003": Y003OutputParser()
}


In [33]:
# use prompts from iv_qns and parsers.format_instructions for formatted qns

# for qn in iv_qns:
#     print(iv_qns[qn])
#     print(parsers[qn].format_instructions())

joint_prompts = {qn: iv_qns[qn] + " " + parsers[qn].format_instructions() for qn in iv_qns}
joint_prompts

In [34]:
# Generate all possible prompts using system prompts and joint_promptsm while keeping the key
all_prompts = [(qn, system_prompt + " " + joint_prompt) for system_prompt in system_prompts for qn, joint_prompt in joint_prompts.items()]
all_prompts

In [35]:
len(all_prompts)

If we are doing 5 repeats for each unique prompt, we have 500 prompts for each LLM to get their dataset to plot

In [36]:
# Copy each prompt in all_prompt 5 times
full_prompt_set = [(qn, prompt) for qn, prompt in all_prompts for _ in range(5)]
len(full_prompt_set)

In [37]:
import asyncio
from ollama import AsyncClient

async def generate_response(qn, prompt, llm, max_retries=5):
    message = {'role': 'user', 'content': prompt}
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = await AsyncClient().chat(model=llm, messages=[message])
            response_content = response['message']['content']
            print(response_content)
            # Parse it using the parser
            parsed_response = parsers[qn].parse(response_content)
            return llm, qn, parsed_response
        except Exception as e:
            print(f"Parsing failed: {e}. Retrying ({retry_count + 1}/{max_retries})...")
            retry_count += 1
            await asyncio.sleep(1)  # Optional: wait a bit before retrying

    return llm, qn, None

In [38]:
await generate_response(all_prompts[0][0], all_prompts[0][1], "llama3:8b")

In [39]:
all_prompts[9]

In [40]:
await generate_response(all_prompts[9][0], all_prompts[9][1], "llama3:8b")

In [41]:
all_prompts[3]

In [42]:
await generate_response(all_prompts[6][0], all_prompts[6][1], "llama3:8b")

In [43]:
import asyncio
from ollama import AsyncClient

async def generate_response_refusal(qn, prompt, llm, max_retries=5):
    message = {'role': 'user', 'content': prompt}
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = await AsyncClient().chat(model=llm, messages=[message, {"role": "system", "content": "Sure thing! Here is my response:"}])
            print(response)
            response_content = response['message']['content']
            # Parse it using the parser
            parsed_response = parsers[qn].parse(response_content)
            return llm, qn, parsed_response
        except Exception as e:
            print(f"Parsing failed: {e}. Retrying ({retry_count + 1}/{max_retries})...")
            retry_count += 1
            await asyncio.sleep(1)  # Optional: wait a bit before retrying

    return llm, qn, None

In [44]:
await generate_response_refusal(all_prompts[6][0], all_prompts[6][1], "llama3:8b")

In [45]:
await generate_response_refusal(all_prompts[3][0], all_prompts[3][1], "llama3:8b" ,1)

In [46]:
# All llms we want to try with
llms = ["qwen2:7b", "llama3:8b"]

In [47]:
from tqdm.asyncio import tqdm

async def generate_all_responses(full_prompt_set, llms, max_retries=5):
    """"
    async gather all_prompts for every llm
    """
    tasks = []
    for llm in llms:
        for qn, prompt in full_prompt_set:
            tasks.append(generate_response(qn, prompt, llm, max_retries))
        
    results = []
    for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Generating responses"):
        result = await f
        results.append(result)
    return results

In [48]:
# select random 10 prompts
import random

random_prompts = random.sample(full_prompt_set, 10)
random_prompts

In [49]:
res = await generate_all_responses(random_prompts, llms)

In [50]:
res

In [51]:
# Convert tuples to a dataframe 
responses_df = pd.DataFrame(res, columns=["llm", "question", "response"])
responses_df

In [52]:
# save to pickle
responses_df.to_pickle("../data/responses_df.pkl")